<a href="https://colab.research.google.com/github/OneFineStarstuff/State-of-the-Art/blob/main/Transformers_for_Time_Series.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn

class TimeSeriesTransformer(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, num_heads, output_dim):
        super(TimeSeriesTransformer, self).__init__()
        self.encoder = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(d_model=hidden_dim, nhead=num_heads), num_layers=num_layers
        )
        self.input_projection = nn.Linear(input_dim, hidden_dim)
        self.output_projection = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = self.input_projection(x)
        x = self.encoder(x)
        return self.output_projection(x)

# Example usage
model = TimeSeriesTransformer(input_dim=1, hidden_dim=64, num_layers=2, num_heads=4, output_dim=1)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

for epoch in range(100):
    x = torch.randn(32, 10, 1)  # Batch of 32, sequence length of 10, 1 feature
    target = torch.randn(32, 10, 1)
    optimizer.zero_grad()
    output = model(x)
    loss = nn.MSELoss()(output, target)
    loss.backward()
    optimizer.step()
    print(f'Epoch {epoch + 1}, Loss: {loss.item():.4f}')